# Personal Project main notebook

In [1]:
# Ignoring warning messages from python
import warnings
warnings.filterwarnings('ignore')

# General use imports
import pandas as pd
import numpy as np

# Visualization imports
import matplotlib.pyplot as plt
import seaborn as sns

# Modules and data
import requests
import acquire
import prep
from vega_datasets import data
from datetime import datetime

# I. Acquire

In [2]:
# Acquiring the data from the csv file and displaying first 5 rows

transactions1 = acquire.get_local_transactions()
transactions1.head()

,Date,Description,Original Description,Amount,Transaction Type,Category,Account Name,Labels,Notes
0,6/29/2022,DEPOSIT@MOBILE,DEPOSIT@MOBILE,206.25,credit,Transfer,Checkarama,NaN,NaN
1,6/28/2022,"MATTHEWS VET SERVICES, P","MATTHEWS VET SERVICES, P",13.20,debit,Veterinary,CREDIT CARD,NaN,NaN
2,6/28/2022,STARBUCKS STORE 24306,STARBUCKS STORE 24306,5.57,debit,Coffee Shops,CREDIT CARD,NaN,NaN
3,6/28/2022,PAYPAL *HULU,PAYPAL *HULU,14.16,debit,Television,CREDIT CARD,NaN,NaN
4,6/28/2022,H-E-B #618,H-E-B #618,20.56,debit,Groceries,CREDIT CARD,NaN,NaN


# II. Clean & Prepare

>## High level exploration before Cleaning and preparation

In [3]:
# Checking the number of rows and columns

transactions1.shape

(7872, 9)

In [4]:
# Comparing the Description and the Original Description columns to see which is worth dropping

transactions1[['Description', 'Original Description']]

,Description,Original Description
0,DEPOSIT@MOBILE,DEPOSIT@MOBILE
1,"MATTHEWS VET SERVICES, P","MATTHEWS VET SERVICES, P"
2,STARBUCKS STORE 24306,STARBUCKS STORE 24306
3,PAYPAL *HULU,PAYPAL *HULU
4,H-E-B #618,H-E-B #618
...,...,...
7867,Ultra Foods Qps,ULTRA FOODS #8761 QPS LOMBARD
7868,Pennys Noodle Shop,PENNY'S NOODLE SHOPQPS CHICAGO
7869,Amazon Music,MUSIC DWNLDS 866-216
7870,Calling Card,CALLINGCARDS/CONFERENC (866)29


In [5]:
# Checking the df nulls

transactions1.isnull().sum()

Date                       0
Description                0
Original Description       0
Amount                     0
Transaction Type           0
Category                   3
Account Name               0
Labels                  7870
Notes                   7862
dtype: int64

In [6]:
# Checking the specific crows of the Category column

transactions1[transactions1['Category'].isnull()]

,Date,Description,Original Description,Amount,Transaction Type,Category,Account Name,Labels,Notes
5996,3/02/2015,Payment,Payment,10.0,credit,NaN,Stafford Loans U.S. DEPARTMENT OF EDUCATION,NaN,NaN
5997,3/02/2015,Pending,Pending,10.0,credit,NaN,Stafford Loans U.S. DEPARTMENT OF EDUCATION,NaN,NaN
5998,2/28/2015,Payment,Payment,40.0,credit,NaN,Stafford Loans U.S. DEPARTMENT OF EDUCATION,NaN,NaN


>## Takeaways
    - The df contains duplicates
    - The df has nulls in great number in labels and Notes columns
    - The df has much fewer nulls in the Category column
    - The Description and the Original Description are duplicates
    - These columns contain special characters
    - There is a date column
>## Actions
    - Remove duplicates
    - Drop Labels, Notes, and Original Description
    - Remove special characters from Description
    - Rename columns for readability
    - Make Account Name and Description's content into lower case
    - Set Date as INDEX

>## Cleaning

In [7]:
# Printing the column names into a list without a line break

a = transactions1.columns.tolist()
print(a,)

['Date', 'Description', 'Original Description', 'Amount', 'Transaction Type', 'Category', 'Account Name', 'Labels', 'Notes']


In [8]:
# Assigning the list of columns to a variable

columns = ['Date', 'Description', 'Original Description', 'Amount', 'Transaction Type', 'Category', 'Account Name', 'Labels', 'Notes']

In [10]:
# Displaying duplicate rows in count and percent

prep.multi_frequency(transactions1, columns)

,num_rows_missing,pct_rows_missing
Date,0,0.000000
Description,0,0.000000
Original Description,0,0.000000
Amount,0,0.000000
Transaction Type,0,0.000000
Category,3,0.038110
Account Name,0,0.000000
Labels,7870,99.974593
Notes,7862,99.872967


In [12]:
# Dropping dupicates and keeping the entries detected and reassigning the df to a new variable

transactions2 = transactions1.drop_duplicates()

In [13]:
transactions2.shape

(7797, 9)

In [14]:
# Dropping columns that are not useful

transactions3 = transactions2.drop(['Account Name', 'Labels', 'Notes', 'Original Description'], axis = 1)

In [15]:
transactions3.head()

,Date,Description,Amount,Transaction Type,Category
0,6/29/2022,DEPOSIT@MOBILE,206.25,credit,Transfer
1,6/28/2022,"MATTHEWS VET SERVICES, P",13.20,debit,Veterinary
2,6/28/2022,STARBUCKS STORE 24306,5.57,debit,Coffee Shops
3,6/28/2022,PAYPAL *HULU,14.16,debit,Television
4,6/28/2022,H-E-B #618,20.56,debit,Groceries


In [16]:
# Checking the nulls in the Category column

transactions3[transactions3['Category'].isnull()]

,Date,Description,Amount,Transaction Type,Category
5996,3/02/2015,Payment,10.0,credit,NaN
5997,3/02/2015,Pending,10.0,credit,NaN
5998,2/28/2015,Payment,40.0,credit,NaN


In [17]:
# Filling out the nulls with the category 'Education'

transactions4 = prep.handle_missing_values(transactions3)

In [18]:
a = transactions4.columns.tolist()
print(a,)

['Date', 'Description', 'Amount', 'Transaction Type', 'Category']


In [19]:
# Assigning columns to a variable

columns4 = ['Date', 'Description', 'Amount', 'Transaction Type', 'Category']

In [20]:
# Checking nulls

prep.multi_frequency(transactions4, columns4)

,num_rows_missing,pct_rows_missing
Date,0,0.0
Description,0,0.0
Amount,0,0.0
Transaction Type,0,0.0
Category,0,0.0


In [22]:
# Checking the number of unique entries in Description column

transactions4['Description'].nunique()

1714

In [23]:
# Checking the number of unique entries in Transaction Type column (Debit and Credit)

transactions4['Transaction Type'].nunique()

2

In [24]:
# Checking the number of unique entries in Category column

transactions4['Category'].nunique()

111

### *Takeaways*

>**Category and description can help define the type of purchases and their categories so I will keep them both**

>**I will rename the columns and clean up Description. Then set date to a datetime type and an index column**

In [25]:
renamed = {
    'Date': 'date',
    'Description': 'description',
    'Amount': 'amount',
    'Transaction Type': 'transaction_type',
    'Category': 'category'
}

In [26]:
transactions5 = transactions4.rename(columns=renamed)
transactions5.head()

,date,description,amount,transaction_type,category
0,6/29/2022,DEPOSIT@MOBILE,206.25,credit,Transfer
1,6/28/2022,"MATTHEWS VET SERVICES, P",13.20,debit,Veterinary
2,6/28/2022,STARBUCKS STORE 24306,5.57,debit,Coffee Shops
3,6/28/2022,PAYPAL *HULU,14.16,debit,Television
4,6/28/2022,H-E-B #618,20.56,debit,Groceries


In [76]:
transactions5[transactions5.category == 'Travel']

,date,description,amount,transaction_type,category
21,6/23/2022,paypal_disney_plus,7.63,debit,Travel
121,5/23/2022,paypal_disney_plus,7.63,debit,Travel
205,4/24/2022,paypal_disney_plus,7.63,debit,Travel
233,4/11/2022,txtag_888_468_9824,10.00,debit,Travel
828,3/12/2020,merry_christmas__all,13.15,debit,Travel
889,2/23/2020,disney,7.63,debit,Travel
916,2/10/2020,princess,64.00,debit,Travel
960,1/23/2020,disney,7.63,debit,Travel
1067,12/23/2019,disney,7.63,debit,Travel
1161,11/19/2019,princess,72.00,debit,Travel


In [31]:
transactions5['description'] = transactions5['description'].str.strip()

In [36]:
transactions5['description'] = transactions5['description'].str.replace('[#,@,&,*, ,]', '_').str.lower()

In [37]:
transactions5

,date,description,amount,transaction_type,category
0,6/29/2022,deposit_mobile,206.25,credit,Transfer
1,6/28/2022,matthews_vet_services__p,13.20,debit,Veterinary
2,6/28/2022,starbucks_store_24306,5.57,debit,Coffee Shops
3,6/28/2022,paypal__hulu,14.16,debit,Television
4,6/28/2022,h-e-b__618,20.56,debit,Groceries
...,...,...,...,...,...
7867,8/07/2010,ultra_foods_qps,17.84,debit,Food & Dining
7868,8/07/2010,pennys_noodle_shop,26.42,debit,Restaurants
7869,8/05/2010,amazon_music,5.00,debit,Music
7870,8/05/2010,calling_card,20.00,debit,Gift


In [38]:
transactions5['description'] = transactions5['description'].str.replace('__', '_')

In [39]:
transactions5

,date,description,amount,transaction_type,category
0,6/29/2022,deposit_mobile,206.25,credit,Transfer
1,6/28/2022,matthews_vet_services_p,13.20,debit,Veterinary
2,6/28/2022,starbucks_store_24306,5.57,debit,Coffee Shops
3,6/28/2022,paypal_hulu,14.16,debit,Television
4,6/28/2022,h-e-b_618,20.56,debit,Groceries
...,...,...,...,...,...
7867,8/07/2010,ultra_foods_qps,17.84,debit,Food & Dining
7868,8/07/2010,pennys_noodle_shop,26.42,debit,Restaurants
7869,8/05/2010,amazon_music,5.00,debit,Music
7870,8/05/2010,calling_card,20.00,debit,Gift


In [50]:
# Displaying the list of unique values of category

transactions5['category'].unique()

array(['Transfer', 'Veterinary', 'Coffee Shops', 'Television',
       'Groceries', 'Shipping', 'Entertainment', 'Interest Income',
       'Restaurants', 'Gas & Fuel', 'Vacation', 'Shopping', 'Fast Food',
       'Charity', 'Travel', 'Business Services', 'Income',
       'Auto Insurance', 'Utilities', 'Alcohol & Bars',
       'Credit Card Payment', 'Newspapers & Magazines', 'Paycheck',
       'Parking', 'Internet', 'Clothing', 'Cash & ATM',
       'Electronics & Software', 'Air Travel', 'Pharmacy', 'Doctor',
       'Mortgage & Rent', 'Health & Fitness', 'Check',
       'Pet Food & Supplies', 'Books & Supplies', 'Uncategorized',
       'Furnishings', 'Gifts & Donations', 'Life Insurance',
       'Auto Payment', 'Rental Car & Taxi', 'Books', 'Laundry', 'Gift',
       'Ride Share', 'Service & Parts', 'Mobile Phone', 'Federal Tax',
       'Taxes', 'Education', 'Misc Expenses', 'Hotel',
       'Bills & Utilities', 'Service Fee', 'ATM Fee', 'Fees & Charges',
       'Tuition', 'Personal Care', 

>**Many categories can be grouped into one**

        - Income and paycheck
        - Rental Car & Taxi and Ride Share and Taxi/Uber
        - Food & Dining and Restaurants
        - Arsene Allowance and Emily Allowance and Uncategorized and Misc Expenses 
        - Cash & ATM and Transfer for Cash Spending and Transfer
        - Late Fee and Fees & Charges and ATM Fee and Service Fee and Spilt with Roommates 
        - Federal Tax and Taxes
        - Health & Fitness and Gym

In [51]:
transactions5.replace(to_replace=['Paycheck'], value='Income', inplace=True)
transactions5.replace(to_replace=['Rental Car & Taxi', 'Ride Share', 'Taxi/Uber'], value='Taxi', inplace=True)
transactions5.replace(to_replace=['Food & Dining'], value='Restaurants', inplace=True)
transactions5.replace(to_replace=['Arsene Allowance',
                                  'Emily Allowance',
                                  'Uncategorized'], value='Miscellaneous', inplace=True)
transactions5.replace(to_replace=['Cash & ATM',
                                  'Transfer for Cash Spending', 'Transfer'], value='Cash', inplace=True)
transactions5.replace(to_replace=['Late Fee',
                                  'Fees & Charges',
                                  'ATM Fee', 'Service Fee', 'Spilt with Roommates'], value='Fees', inplace=True)
transactions5.replace(to_replace=['Federal Tax'], value='Taxes', inplace=True)
transactions5.replace(to_replace=['Health & Fitness', 'Gym'], value='Health_and_Fitness', inplace=True)

In [62]:
transactions5['category'].nunique()

97

In [61]:
transactions5['category'].unique()

array(['Cash', 'Veterinary', 'Coffee Shops', 'Television', 'Groceries',
       'Shipping', 'Entertainment', 'Interest Income', 'Restaurants',
       'Gas & Fuel', 'Vacation', 'Shopping', 'Fast Food', 'Charity',
       'Travel', 'Business Services', 'Income', 'Auto Insurance',
       'Utilities', 'Alcohol & Bars', 'Credit Card Payment',
       'Newspapers & Magazines', 'Parking', 'Internet', 'Clothing',
       'Electronics & Software', 'Air Travel', 'Pharmacy', 'Doctor',
       'Mortgage & Rent', 'Health_and_Fitness', 'Check',
       'Pet Food & Supplies', 'Books & Supplies', 'Miscellaneous',
       'Furnishings', 'Gifts & Donations', 'Life Insurance',
       'Auto Payment', 'Taxi', 'Books', 'Laundry', 'Gift',
       'Service & Parts', 'Mobile Phone', 'Taxes', 'Education',
       'Misc Expenses', 'Hotel', 'Bills & Utilities', 'Fees', 'Tuition',
       'Personal Care', 'Hobbies', 'Financial', 'Hair', 'Loan Payment',
       'Movies & DVDs', 'Dentist', 'Kids Activities', 'Loans',
       'S

>**More groupings are possible**

        - Entertainment and Hobbies and Movies & DVDs and Amusement and Music and Art and CSA Membership and Piano Lessons and Sports and Television and Sporting Goods
        - Kids Activities and Babysitter & Daycare and Gifts & Donations and Charity and Arsene and Gift and Beatrice
        - Hide from Budgets & Trends and Fees and Business Services and Bank Fee and Finance Charge and Financial and Check and Shipping and Printing and Electronics & Software
        - Air Travel and Vacation and Hotel
        - Pharmacy and Dentist and Doctor and Counseling and Health Insurance
        - Auto & Transport and Service & Parts and Parking
        - Lawn & Garden and Home Improvement and Home Supplies and Home Services and Home and Furnishings
        - Alcohol & Bars and Restaurants and Coffee Shops and Fast Food
        - Books and Newspapers & Magazines and Books & Supplies
        - Bills & Utilities and Utilities and Calling Card and Mobile Phone and Home Phone and Internet
        - Pet Food & Supplies and Veterinary
        - Clothing and Shopping and Hair and Personal Care and Laundry
        - Tuition and Education
        - Misc Expenses and Miscellaneous
        - Loan Payment and Loans and Loan Interest and Loan Principal and Student Loan

In [72]:
transactions5.replace(to_replace=['Entertainment',
                                  'Hobbies',
                                  'Movies & DVDs',
                                  'Amusement',
                                  'Music',
                                  'Art',
                                  'CSA Membership',
                                  'Piano Lessons',
                                  'Sports',
                                  'Television', 'Sporting Goods'],
                                  value='Entertainment_and_Sports', inplace=True)
transactions5.replace(to_replace=['Kids Activities',
                                  'Babysitter & Daycare',
                                  'Gifts & Donations',
                                  'Charity',
                                  'Arsene',
                                  'Gift', 'Beatrice'], value='Donations_and_Gifts', inplace=True)
transactions5.replace(to_replace=['Hide from Budgets & Trends',
                                  'Business Services',
                                  'Bank Fee',
                                  'Legal',
                                  'Finance Charge',
                                  'Financial', 'Check',
                                  'Electronics & Software',
                                  'Printing', 'Shipping'], value='Services_Fees', inplace=True)
transactions5.replace(to_replace=['Air Travel', 'Hotel'], value='Vacation', inplace=True)
transactions5.replace(to_replace=['Pharmacy',
                                  'Dentist',
                                  'Doctor',
                                  'Counseling',
                                  'Health Insurance'], value='Health', inplace=True)
transactions5.replace(to_replace=['Auto & Transport',
                                  'Service & Parts', 'Parking'], value='Auto_Fees', inplace=True)
transactions5.replace(to_replace=['Lawn & Garden',
                                  'Home Improvement',
                                  'Home Supplies',
                                  'Home Services',
                                  'Home', 'Furnishings'], value='Home_Improvement', inplace=True)
transactions5.replace(to_replace=['Alcohol & Bars',
                                  'Restaurants',
                                  'Coffee Shops', 'Fast Food'], value='Outings', inplace=True)
transactions5.replace(to_replace=['Books',
                                  'Newspapers & Magazines',
                                  'Books & Supplies'], value='Books_and_Newspapers', inplace=True)
transactions5.replace(to_replace=['Bills & Utilities',
                                  'Utilities',
                                  'Calling Card',
                                  'Mobile Phone',
                                  'Home Phone', 'Internet'], value='Bills_and_Utilities', inplace=True)
transactions5.replace(to_replace=['Pet Food & Supplies',
                                  'Veterinary'], value='Pet', inplace=True)
transactions5.replace(to_replace=['Clothing',
                                  'Shopping',
                                  'Hair',
                                  'Personal Care',
                                  'Laundry'], value='Care_and_Shopping', inplace=True)
transactions5.replace(to_replace=['Tuition'], value='Education', inplace=True)
transactions5.replace(to_replace=['Misc Expenses'], value='Miscellaneous', inplace=True)
transactions5.replace(to_replace=['Loan Payment',
                                  'Loan Interest',
                                  'Loan Principal',
                                  'Student Loan'], value='Loans', inplace=True)

In [73]:
transactions5['category'].nunique()

39

In [74]:
transactions5['category'].unique()

array(['Cash', 'Pet', 'Outings', 'Entertainment_and_Sports', 'Groceries',
       'Services_Fees', 'Interest Income', 'Gas & Fuel', 'Vacation',
       'Care_and_Shopping', 'Donations_and_Gifts', 'Travel', 'Income',
       'Auto Insurance', 'Bills_and_Utilities', 'Credit Card Payment',
       'Books_and_Newspapers', 'Auto_Fees', 'Health', 'Mortgage & Rent',
       'Health_and_Fitness', 'Miscellaneous', 'Home_Improvement',
       'Life Insurance', 'Auto Payment', 'Taxi', 'Taxes', 'Education',
       'Fees', 'Loans', 'Spa & Massage', 'Public Transportation',
       'Office Supplies', 'Investments', 'Arts', 'Reimbursement',
       'State Tax', 'Wedding', 'Deposit'], dtype=object)

In [80]:
transactions5.replace(to_replace=['Auto Insurance', 'Auto Payment'], value='Auto_Payments', inplace=True)
transactions5.replace(to_replace=['Spa & Massage'], value='Care_and_Shopping', inplace=True)
transactions5.replace(to_replace=['Office Supplies',
                                  'Books_and_Newspapers'], value='Books_and_Supplies', inplace=True)
transactions5.replace(to_replace=['Arts'], value='Entertainment_and_Sports', inplace=True)
transactions5.replace(to_replace=['State Tax'], value='Taxes', inplace=True)
transactions5.replace(to_replace=['Vacation'], value='Travel', inplace=True)
transactions5.replace(to_replace=['Gas & Fuel'], value='Gas', inplace=True)

In [81]:
transactions5['category'].nunique()

33

In [82]:
transactions5['category'].unique()

array(['Cash', 'Pet', 'Outings', 'Entertainment_and_Sports', 'Groceries',
       'Services_Fees', 'Interest Income', 'Gas', 'Travel',
       'Care_and_Shopping', 'Donations_and_Gifts', 'Income',
       'Auto_Payments', 'Bills_and_Utilities', 'Credit Card Payment',
       'Books_and_Supplies', 'Auto_Fees', 'Health', 'Mortgage & Rent',
       'Health_and_Fitness', 'Miscellaneous', 'Home_Improvement',
       'Life Insurance', 'Taxi', 'Taxes', 'Education', 'Fees', 'Loans',
       'Public Transportation', 'Investments', 'Reimbursement', 'Wedding',
       'Deposit'], dtype=object)

In [ ]:
# Setting the date column to a datetime type

transactions4.date = pd.to_datetime(transactions4.date)

In [ ]:
transactions5 = transactions4.set_index('date').sort_index()

In [ ]:
transactions5.head()

# Explore